In [51]:
%pip install pydantic
%pip install pydantic[email]

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [52]:
from pydantic import BaseModel, EmailStr, Field, ValidationError, field_validator
from typing import Optional

### Modelo Pydantic representando um formulário de registro do usuário e funções de validação.

In [53]:
class FormRegistro(BaseModel):
    """
    Valida os campos:
    Username, deve ter 3-20 caracteres
    Email, formato válido (EmailStr)
    Password, pelo menos 8 caracteres e conter 1 dígito
    Idade, Opcional, mas se fornecida, >= 18
    """
    username: str
    email: EmailStr = Field(
        examples=["felipe@ai.com"],
        description="The email address of the user", # Descrição do campo email
        frozen=True,
    )
    password: str
    idade: Optional[int] = None

    # Validação customizada para o username
    @field_validator("username")
    def validate_username(cls, value):
        if len(value) < 3 or len(value) > 20: #Verifica o tamanho de username
            raise ValueError("Username deve ter 3-20 caracteres.") #Tratamento de erros
        if not value.isalnum():
            raise ValueError("Username deve ser alphanumerico.")
        return value

    # Validação customizada para password
    @field_validator("password")
    def validate_password(cls, value):
        if len(value) < 8: #Verifica o tamanho de password
            raise ValueError("Password deve ter pelo menos 8 caracteres.") #Tratamento de erros
        if not any(c.isdigit() for c in value):
            raise ValueError("Password deve conter pelo menos um dígito.")
        return value
    
    # Validação customizada para idade
    @field_validator("idade")
    def validate_age(cls, value):
        if value is not None and value < 18: #Verifica o valor de idade
            raise ValueError("Idade deve ser 18 ou mais.") #Tratamento de erros
        return value


### Exemplo de uso:

In [54]:
def submit_registration(form_data: dict):
    try:
        registration = FormRegistro(**form_data)
        print("✅ Registro válido!")
        print(registration.model_dump_json(indent=2))
    except ValidationError as e:
        print("❌ Erros de validação encontrados:")
        for error in e.errors():
            field = error["loc"][0]  # Obtém o nome do campo com erro
            error_msg = error["msg"]  # Mensagem de erro do Pydantic
            submitted_value = form_data.get(field, "não fornecido")  # Valor submetido
            
            # Mensagem personalizada
            print(f"  - Campo '{field}':")
            print(f"     Valor inválido: '{submitted_value}'")
            print(f"     Problema: {error_msg}\n")

### Casos teste

In [55]:
if __name__ == "__main__":
    # Caso 1: Dados válidos
    valid_data = {
        "username": "alice123",
        "email": "alice@example.com",
        "password": "secure123",
        "age": 25
    }
    submit_registration(valid_data)

    # Case 2: Dados inválidos (password curta, idade abaixo de 18)
    invalid_data = {
        "username": "bob!",
        "email": "bob@example.com",
        "password": "short",
        "age": 17
    }
    submit_registration(invalid_data)

✅ Registro válido!
{
  "username": "alice123",
  "email": "alice@example.com",
  "password": "secure123",
  "idade": null
}
❌ Erros de validação encontrados:
  - Campo 'username':
     Valor inválido: 'bob!'
     Problema: Value error, Username deve ser alphanumerico.

  - Campo 'password':
     Valor inválido: 'short'
     Problema: Value error, Password deve ter pelo menos 8 caracteres.

